In [1]:
import pandas as pd
import numpy as np

In [2]:
data_source = "https://archive.ics.uci.edu/ml/machine-learning-databases/cmc/cmc.data"

names = ["Wife's age", "Wife's education", "Husband's education", "Number of children ever born", "Wife's religion", 
"Wife's now working?", "Husband's occupation", "Standard-of-living index", "Media exposure", "Contraceptive method used"]

dataframe = pd.read_csv(data_source, names=names)

In [3]:
for column in range(0,10):
    if column not in [0,3]:
    #if column not in [0,3,4,5,8]:
        dataframe[names[column]] = dataframe[names[column]].astype("category")
    
#dataframe.dtypes
# y_data = df["Contraceptive method used"].values.reshape(-1,1)

# y_data[:5]

In [4]:
y_data = dataframe.pop("Contraceptive method used")

# dataframe.pop("Wife's age")
x_data = dataframe
x_data.insert(0, "bias", 1)

In [5]:
x_data = pd.get_dummies(x_data).as_matrix()
y_data = pd.get_dummies(y_data).as_matrix()

In [6]:
x_data[1]

array([  1.,  45.,  10.,   1.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,
         0.,   1.,   0.,   1.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,
         1.,   1.,   0.])

In [7]:
from sklearn import preprocessing as p

min_max_scaler=p.MinMaxScaler()
x_data[:, 1:3] = min_max_scaler.fit_transform(x_data[:,1:3])

In [8]:
idx=np.random.randint(y_data.shape[0],size=int(y_data.shape[0]))
training_idx = idx[:int(y_data.shape[0]*0.8)]
test_idx = idx[int(y_data.shape[0]*0.8):]

x_training, x_test = x_data[training_idx], x_data[test_idx]
y_training, y_test = y_data[training_idx], y_data[test_idx]

In [9]:
y_training.shape

(1178, 3)

In [10]:
import tensorflow as tf

# x = x_training.T

W = tf.Variable(tf.zeros([len(x_data[0]), len(y_data[0])]))

# W = tf.Variable(tf.random_uniform([len(y_training[0]),len(x)], -1.0, 1.0))

# X = tf.placeholder(tf.float32)
# Y = tf.placeholder(tf.float32)
                
X = tf.placeholder("float", [None, len(x_data[0])])
Y = tf.placeholder("float", [None, len(y_data[0])])


hypothesis = tf.nn.softmax(tf.matmul(X, W))

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), reduction_indices=1))

W_val = []
cost_val = []

a = tf.Variable(0.001)
optimizer = tf.train.GradientDescentOptimizer(0.001)
train = optimizer.minimize(cost)

init = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init)

#y_training = y_training.T

for step in range(5000000):
    sess.run(train, feed_dict={X:x_training, Y:y_training})
    if step % 500000 == 0:
        print (step, sess.run(cost, feed_dict={X:x_training, Y:y_training}), sess.run(W))

0 1.0985 [[  9.19637896e-05  -1.13469214e-04   2.15053606e-05]
 [  6.04431443e-05  -4.08070664e-05  -1.96360998e-05]
 [  1.18845437e-05  -1.71371084e-05   5.25253927e-06]
 [  4.24448262e-05  -3.31069677e-05  -9.33786305e-06]
 [  4.27277919e-05  -3.96151809e-05  -3.11262647e-06]
 [  1.64119883e-05  -4.72552565e-05   3.08432354e-05]
 [ -9.62083777e-06   6.50819493e-06   3.11261010e-06]
 [  8.48896434e-06  -8.48897173e-07  -7.64006927e-06]
 [  3.45217886e-05  -3.08432500e-05  -3.67855341e-06]
 [  3.08432354e-05  -5.40464280e-05   2.32031653e-05]
 [  1.81097748e-05  -2.77306408e-05   9.62080958e-06]
 [  8.48894388e-07  -5.09338133e-06   4.24448081e-06]
 [  9.11148963e-05  -1.08375832e-04   1.72608761e-05]
 [  3.02773060e-05  -2.40520803e-05  -6.22524749e-06]
 [  6.16864709e-05  -8.94171317e-05   2.77306062e-05]
 [  7.92302762e-06   1.04697183e-05  -1.83927677e-05]
 [  3.82003418e-05  -4.24448408e-05   4.24447808e-06]
 [  4.07470325e-05  -7.72496132e-05   3.65025480e-05]
 [  5.09337860e-06 

KeyboardInterrupt: 

In [11]:
hypothesis_value=sess.run(hypothesis,feed_dict={X:x_test})
result=[np.argmax(predict) == np.argmax(original_value) for predict, original_value in zip(hypothesis_value,y_test)]

sum(result)/len(y_test)

0.52203389830508473